In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import pandas as pd

In [5]:
train_path = '/kaggle/input/adobetraindata/behaviour_simulation_train.csv'
test_path = '/kaggle/input/inter-iit-mid-prep-adobe/problem_1_test_dataset/behaviour_simulation_test_company.xlsx'

In [6]:
test_dataset = pd.read_excel(test_path)

In [7]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                10000 non-null  int64 
 1   date              10000 non-null  object
 2   content           10000 non-null  object
 3   username          10000 non-null  object
 4   media             10000 non-null  object
 5   inferred company  10000 non-null  object
dtypes: int64(1), object(5)
memory usage: 468.9+ KB


In [8]:
train_dataset = pd.read_csv(train_path)

In [25]:
import matplotlib.pyplot as plt

In [27]:
Photo_data = train_dataset[train_dataset['media'].apply(lambda x: x.startswith('[Photo'))].copy().reset_index(drop=True)

In [28]:
img_links = Photo_data['media'].apply(lambda x:x.split("Photo(previewUrl")[1].split("'")[1])

In [29]:
import pickle
with open('/kaggle/input/image-data-adobe/image_data.pkl','rb') as f:
    image_data = pickle.load(f)

In [30]:
import numpy as np

In [31]:
valid_idxs = []
for i,_ in enumerate(image_data):
    if _ is not None:
        valid_idxs.append(i)
Photo_data = Photo_data.iloc[valid_idxs].reset_index(drop=True)
image_data = np.array(image_data)[valid_idxs]


In [32]:
from sklearn.preprocessing import LabelEncoder

# Define custom bins
bins = [0, 100, 500, 1000, 2000, 5000, 10000, 20000, 50000, 100000, 500000]
labels = ['0-100', '101-500', '501-1k', '1k-2k', '2k-5k', '5k-10k', '10k-20k', '20k-50k', '50k-100k', '100k+']

# Bin the likes into categories
Photo_data['likes_binned'] = pd.cut(Photo_data['likes'], bins=bins, labels=labels, include_lowest=True)

# Convert bin labels to numeric categories
le = LabelEncoder()
y = le.fit_transform(Photo_data['likes_binned'])

In [33]:
Photo_data['likes_binned'].value_counts()

likes_binned
0-100       124394
101-500      44644
501-1k       16248
1k-2k         9279
2k-5k         6415
5k-10k        1991
10k-20k        934
20k-50k        623
50k-100k       101
100k+           41
Name: count, dtype: int64

In [34]:
# model = LikesPredictionModel()

In [35]:
# model.load_state_dict(torch.load('/kaggle/input/cnn-adobe/image_likes_model.pth'))

In [36]:
# len(Photo_data['likes'])

In [37]:
from sklearn.model_selection import StratifiedGroupKFold,StratifiedKFold

In [39]:
splitter = StratifiedKFold(n_splits=10, random_state = 42,shuffle=True)
split = splitter.split(Photo_data, y)
train_inds, test_inds = next(split)
Photo_data_sub, y_sub, image_data_sub = Photo_data.copy().iloc[test_inds].reset_index(drop=True), np.array(y[test_inds]), image_data[test_inds]

In [50]:
# splitter = StratifiedGroupKFold(n_splits=4, random_state = 42,shuffle=True)
# split = splitter.split(Photo_data, y,groups=Photo_data['inferred company'])
# train_inds, test_inds = next(split)
# # from imblearn.under_sampling import RandomUnderSampler
# train_images, train_likes = image_data[train_inds], np.array(y[train_inds])
# # rus = RandomUnderSampler(sampling_strategy='majority',random_state=42)
# # X_res, y_res = rus.fit_resample(train_images, train_likes)
# # X_res = X_res.squeeze()
# # y_res = y_res.squeeze()
# val_images, val_likes = image_data[test_inds], np.array(y[test_inds])
splitter = StratifiedGroupKFold(n_splits=3, random_state = 42,shuffle=True)
split = splitter.split(Photo_data_sub, y_sub,groups=Photo_data_sub['inferred company'])
train_inds, test_inds = next(split)
# from imblearn.under_sampling import RandomUnderSampler
train_images, train_likes, train_text = image_data_sub[train_inds], np.array(y_sub[train_inds]), Photo_data_sub.iloc[train_inds].reset_index(drop=True)['content']
# rus = RandomUnderSampler(sampling_strategy='majority',random_state=42)
# X_res, y_res = rus.fit_resample(train_images, train_likes)
# X_res = X_res.squeeze()
# y_res = y_res.squeeze()
val_images, val_likes, val_text = image_data_sub[test_inds], np.array(y_sub[test_inds]), Photo_data_sub.iloc[test_inds].reset_index(drop=True)['content']

In [51]:
len(train_likes)/len(val_likes)

3.2295928910932012

In [62]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
from PIL import Image
import io
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from transformers import CLIPProcessor, CLIPModel

# Load CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

class ImageTextLikesDataset(Dataset):
    def __init__(self, image_data, text_data, likes):
        self.image_data = image_data
        self.text_data = text_data
        self.likes = likes
    
    def __len__(self):
        return len(self.image_data)
    
    def __getitem__(self, idx):
        image = Image.open(io.BytesIO(self.image_data[idx])).convert('RGB')
        text = self.text_data[idx]
        return image, text, torch.tensor(self.likes[idx], dtype=torch.long)

class VLMEmbeddingModel(nn.Module):
    def __init__(self):
        super(VLMEmbeddingModel, self).__init__()
        self.clip = clip_model
    
    def forward(self, images, texts):
        inputs = clip_processor(text=texts, images=images, return_tensors="pt", padding=True, truncation=True)
        inputs = {k: v.to(self.clip.device) for k, v in inputs.items()}
        outputs = self.clip(**inputs)
        image_features = outputs.image_embeds
        text_features = outputs.text_embeds
        combined_features = torch.cat((image_features, text_features), dim=1)
        return combined_features

def collate_fn(batch):
    images, texts, likes = zip(*batch)
    return list(images), list(texts), torch.stack(likes)

In [63]:
def train_model(model, train_loader, val_loader, num_epochs=2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    model.to(device)
    
#     for epoch in range(num_epochs):
#     print(f'Epoch {epoch+1}/{num_epochs} running')
    model.eval()
    train_embeddings = []
    train_likes = []
    with torch.no_grad():
        for images, texts, likes in tqdm(train_loader):
            likes = likes.to(device)
            outputs = model(images, texts)
            train_embeddings.append(outputs.detach().cpu().numpy())
            train_likes.append(likes.cpu().numpy())

    train_embeddings = np.vstack(train_embeddings)
    train_likes = np.concatenate(train_likes)

    model.eval()
    val_embeddings = []
    val_likes = []

    with torch.no_grad():
        for images, texts, likes in tqdm(val_loader):
            likes = likes.to(device)
            outputs = model(images, texts)
            val_embeddings.append(outputs.detach().cpu().numpy())
            val_likes.append(likes.cpu().numpy())

    val_embeddings = np.vstack(val_embeddings)
    val_likes = np.concatenate(val_likes)

    # Train Random Forest on the embeddings
    rf_classifier = RandomForestClassifier(max_depth=10, class_weight='balanced')
    rf_classifier.fit(train_embeddings, train_likes)

    train_predictions = rf_classifier.predict(train_embeddings)
    val_predictions = rf_classifier.predict(val_embeddings)

    train_accuracy = accuracy_score(train_likes, train_predictions)
    val_accuracy = accuracy_score(val_likes, val_predictions)

    print(f'Train Accuracy: {train_accuracy:.4f}, Val Accuracy: {val_accuracy:.4f}')
    
    return model, rf_classifier

In [64]:
# Assuming you have these variables: train_images, val_images, train_likes, val_likes, Photo_data

train_dataset = ImageTextLikesDataset(train_images, train_text, train_likes)
val_dataset = ImageTextLikesDataset(val_images, val_text, val_likes)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, collate_fn=collate_fn)

# Initialize the model
model = VLMEmbeddingModel()

In [65]:
# Train the model
trained_model, rf_classifier = train_model(model, train_loader, val_loader, num_epochs=2)

Using device: cuda


  0%|          | 0/489 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(tru

Train Accuracy: 0.9129, Val Accuracy: 0.4348


In [ ]:
def predict_likes(model, rf_classifier, image_dataloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    all_embeddings = []
    all_likes = []
    
    with torch.no_grad():
        for images, texts, likes in tqdm(image_dataloader, desc="Predicting"):
            outputs = model(images, texts)
            all_embeddings.append(outputs.detach().cpu().numpy())
            all_likes.append(likes.numpy())
    
    all_embeddings = np.vstack(all_embeddings)
    all_likes = np.concatenate(all_likes)
    
    predictions = rf_classifier.predict(all_embeddings)
    return predictions, all_likes

In [ ]:
# Make predictions
val_predictions, val_likes = predict_likes(trained_model, rf_classifier, val_loader)
train_predictions, train_likes = predict_likes(trained_model, rf_classifier, train_loader)

# Evaluate the classifier
val_accuracy = accuracy_score(val_likes, val_predictions)
train_accuracy = accuracy_score(train_likes, train_predictions)

print(f"Validation Accuracy: {val_accuracy}")
print(f"Train Accuracy: {train_accuracy}")

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
# import numpy as np
# from PIL import Image
# import io
# from sklearn.model_selection import train_test_split
# import timm
# from tqdm.auto import tqdm
# import os
# from functools import lru_cache

# class OptimizedImageLikesDataset(Dataset):
#     def __init__(self, image_data, likes, transform=None, cache_size=1000):
#         self.image_data = np.array(image_data, dtype=object)
#         self.likes = np.array(likes)
#         self.transform = transform
#         self.cache_size = cache_size

#     def __len__(self):
#         return len(self.likes)

#     @lru_cache(maxsize=1000)
#     def __getitem__(self, idx):
#         image = Image.open(io.BytesIO(self.image_data[idx])).convert('RGB')
#         if self.transform:
#             image = self.transform(image)
#         return image, torch.tensor(self.likes[idx], dtype=torch.long)

# def worker_init_fn(worker_id):
#     worker_info = torch.utils.data.get_worker_info()
#     dataset = worker_info.dataset
#     worker_id = worker_info.id
#     num_workers = worker_info.num_workers
#     per_worker = len(dataset) // num_workers
#     dataset.image_data = np.memmap('image_data.memmap', dtype='uint8', mode='r', 
#                                    shape=(len(dataset.likes), len(dataset.image_data[0])),
#                                    offset=worker_id * per_worker * dataset.image_data[0].nbytes,
#                                    length=per_worker * dataset.image_data[0].nbytes)

# class LikesPredictionModel(nn.Module):
#     def __init__(self):
#         super(LikesPredictionModel, self).__init__()
#         self.efficientnet = timm.create_model('efficientnet_b0', pretrained=True)
#         self.fc1 = nn.Linear(1000, 512)
#         self.dropout = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(512, 10)
#         self.bn1 = nn.BatchNorm1d(1000)
#         self.bn2 = nn.BatchNorm1d(512)

#     def forward(self, x):
#         x = self.efficientnet(x)
#         x = torch.relu(self.fc1(self.bn1(self.dropout(torch.relu(x)))))
#         x = self.bn2(self.dropout(x))
#         x = self.fc2(x)
#         return x.squeeze()

# def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=2):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Using device: {device}")
#     model.to(device)
    
#     for epoch in range(num_epochs):
#         print(f'Epoch {epoch+1}/{num_epochs}')
#         model.train()
#         train_loss = 0.0
#         train_correct = 0
#         train_total = 0

#         for images, likes in tqdm(train_loader, desc="Training"):
#             images, likes = images.to(device), likes.to(device)
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, likes)
#             loss.backward()
#             optimizer.step()
            
#             train_loss += loss.item()
#             _, predicted = outputs.max(1)
#             train_total += likes.size(0)
#             train_correct += predicted.eq(likes).sum().item()

#         train_loss /= len(train_loader)
#         train_acc = train_correct / train_total
        
#         model.eval()
#         val_loss = 0.0
#         val_correct = 0
#         val_total = 0

#         with torch.no_grad():
#             for images, likes in tqdm(val_loader, desc="Validation"):
#                 images, likes = images.to(device), likes.to(device)
#                 outputs = model(images)
#                 loss = criterion(outputs, likes)
                
#                 val_loss += loss.item()
#                 _, predicted = outputs.max(1)
#                 val_total += likes.size(0)
#                 val_correct += predicted.eq(likes).sum().item()

#         val_loss /= len(val_loader)
#         val_acc = val_correct / val_total

#         print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

#     return model

In [ ]:
# len(train_images[0])

In [ ]:
# # Assume image_data is a list of binary image data and likes is a list of corresponding like counts
# # image_data, likes = load_data()  # You need to implement this function to load your data

# # Split the data
# # train_images, val_images, train_likes, val_likes = train_test_split(image_data, likes, test_size=0.2, random_state=42)

# # Define transformations
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# # Create datasets
# train_dataset = OptimizedImageLikesDataset(train_images, train_likes, transform=transform)
# val_dataset = OptimizedImageLikesDataset(val_images, val_likes, transform=transform)

# # Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, worker_init_fn=worker_init_fn)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, worker_init_fn=worker_init_fn)

In [ ]:
# # Initialize the model
# model = LikesPredictionModel()

# # Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# # Train the model
# trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=2)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
# from efficientnet_pytorch import EfficientNet
import numpy as np
from PIL import Image
import io
from sklearn.model_selection import train_test_split
import timm
from tqdm.auto import tqdm

class ImageLikesDataset(Dataset):
    def __init__(self, image_data, likes, transform=None):
        self.image_data = image_data
        self.likes = likes
        self.transform = transform

    def __len__(self):
        return len(self.image_data)

    def __getitem__(self, idx):
        image = Image.open(io.BytesIO(self.image_data[idx])).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(self.likes[idx], dtype=torch.long)

class LikesPredictionModel(nn.Module):
    def __init__(self):
        super(LikesPredictionModel, self).__init__()
        self.effnet = timm.create_model('efficientnet_b0', pretrained=True)
#         self.model = nn.Sequential(
#             nn.Linear(1000,512),
#             nn.BatchNorm1d(512),
#             nn.ReLU(),
#             nn.Linear(512,512),
#             nn.BatchNorm1d(512),
#             nn.ReLU(),
#             nn.Linear(512,512),
#             nn.BatchNorm1d(512),
#             nn.ReLU(),
#             nn.Linear(512,10)
#         )
#         for p in self.effnet.parameters
#         self.fc1 = nn.Linear(1000, 512)
# #         self.dropout = nn.Dropout(0.7)
#         self.fc2 = nn.Linear(512, 10)
#         self.bn1 = nn.BatchNorm1d(1000)
#         self.bn2 = nn.BatchNorm1d(512)

    def forward(self, x):
        x = self.effnet(x)
#         x = torch.relu((self.bn1(self.fc1(x))))
#         x = torch.relu((self.bn1(self.fc1(x))))
#         x = self.fc2(x)
#         x = self.bn2((x))
#         x = self.model(x)
        return x.squeeze()

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    model.to(device)
    
    for epoch in range(num_epochs):
        print(f'epoch {epoch} running')
        model.train()
        train_loss = 0.0
        y=[]
        preds=[]
        i=0
        for images, likes in (tqdm(train_loader)):
            if i%1000==0 : print(f'batch {i} running')
            i+=1
            y.extend(likes)
            images, likes = images.to(device), likes.to(device)
            outputs = model(images)
            preds.extend(np.argmax(outputs.detach().cpu(),axis=1))
            loss = criterion(outputs, likes)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_acc = (np.array(y)==np.array(preds)).mean()
        
        model.eval()
        val_loss = 0.0
        y=[]
        preds=[]
        with torch.no_grad():
            for images, likes in tqdm(val_loader):
                y.extend(likes)
                images, likes = images.to(device), likes.to(device)
                outputs = model(images)
                preds.extend(np.argmax(outputs.detach().cpu(),axis=1))
                loss = criterion(outputs, likes)
                val_loss += loss.item()
            val_acc = (np.array(y)==np.array(preds)).mean()

        val_loss /= len(val_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    return model



In [ ]:
# Assume image_data is a list of binary image data and likes is a list of corresponding like counts
# image_data, likes = load_data()  # You need to implement this function to load your data

# Split the data
# train_images, val_images, train_likes, val_likes = train_test_split(image_data, likes, test_size=0.2, random_state=42)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets
train_dataset = ImageLikesDataset(train_images, train_likes, transform=transform)
val_dataset = ImageLikesDataset(val_images, val_likes, transform=transform)

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Initialize the model
model = LikesPredictionModel()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
type(image_data[0])

In [ ]:
# # Train the model
# trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

# # Save the model
# torch.save(trained_model.state_dict(), 'image_likes_model.pth')

In [ ]:
def predict_likes(model, image_dataloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    preds = []
    likess =[]
    with torch.no_grad():
        for images, likes in tqdm(image_dataloader):
            likess.extend(likes)
            images, likes = images.to(device), likes.to(device)
            outputs = model(images)
#             loss = criterion(outputs, likes)
#             val_loss += loss.item() * images.size(0)
            preds.extend((outputs.detach().cpu()))

    return np.array(preds,dtype='f') ,np.array(likess,dtype='f')  # Convert back from log scale

# Example usage
# model = LikesPredictionModel()
# model.load_state_dict(torch.load('image_likes_model.pth'))
# predicted_likes = predict_likes(model, some_image_data)
# print(f"Predicted likes: {predicted_likes:.0f}")

In [ ]:
predicted_likes_v,likes_v = predict_likes(model, val_loader)
predicted_likes_t,likes_t = predict_likes(model, train_loader)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

# Instantiate the DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier()

# Train the classifier
dt_classifier.fit(predicted_likes_t, likes_t)

# Make predictions on the validation data
val_predictions = dt_classifier.predict(predicted_likes_v)

# Evaluate the classifier
accuracy = accuracy_score(likes_v, val_predictions)
mse = mean_squared_error(likes_v, val_predictions)

print(f"Validation Accuracy: {accuracy}")
print(f"Validation MSE: {mse}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

# Instantiate the RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Train the classifier
rf_classifier.fit(predicted_likes_t, likes_t)

# Make predictions on the validation data
val_predictions = rf_classifier.predict(predicted_likes_v)

# Evaluate the classifier
accuracy = accuracy_score(likes_v, val_predictions)
mse = mean_squared_error(likes_v, val_predictions)

print(f"Validation Accuracy: {accuracy}")
print(f"Validation MSE: {mse}")

In [ ]:
# np.array(np.unique(predicted_likes, return_counts=True)).T

In [ ]:
# np.array(np.unique(likes, return_counts=True)).T

In [ ]:
# criterion(np.clip(predicted_likes,0,500000), likes)

In [ ]:
# np.clip()